In [1]:
from scipy import linalg
import numpy as np
import pandas as pd

In [2]:
c = np.array([-2,4,0,0,0])
b = np.array([4,2,2])
A = np.array([[1,1,1,0,0],
             [-1,1,0,1,0],
             [1,-1,0,0,1]])

In [3]:
def prep_data(basis, non_basis, sol, cB):
  xB = [f"x{el + 1}={round(s,3)}" for el,s in zip(basis, sol)]
  xN = [f"x{el + 1}=0" for el in non_basis]
  cost = round(sol @ cB, 3) # calculate cost
  return xB, xN, cost

In [4]:
basis_indices = [0,1,4]
k = 0
temp = list()

while True:
  k += 1
  non_basis_indices = list(set(range(5)).difference(set(basis_indices)))
  non_basis_indices.sort() # ensure indices are in ascending order

  cn = c[[non_basis_indices]]
  cb = c[[basis_indices]]

  B = A[:, basis_indices]
  B_inv = linalg.inv(B)
  sol = B_inv @ b
  xn = np.zeros(2)

  reduced_costs = cn -cb @ B_inv @ A[:,non_basis_indices]

  if np.all(reduced_costs >=0): # optimal solution found
    xB, xN, cost = prep_data(basis_indices, non_basis_indices, sol, cb)
    temp.append([xB, xN, cb, cn, cost, reduced_costs, None, None, None, k])
    break

  # Bland's Rule: find smallest index with negative reduced cost
  new_entry_index = np.array(non_basis_indices)[np.where(reduced_costs<0)][0]
  db = -B_inv @ A[:, new_entry_index]

  if np.all(db >= 0): # unbounded optimum found
    xB, xN, cost = prep_data(basis_indices, non_basis_indices, sol, cb)
    temp.append([xB, xN, cb, cn, cost, reduced_costs, db, None, None, k])
    break

  thetas = -sol/db
  theta_star = min(thetas[db<0])

  # Bland's Rule: replace smallest index with minimal theta
  exit_index = np.array(basis_indices)[(thetas == theta_star)&(db<0)][0]
  
  xB, xN, cost = prep_data(basis_indices, non_basis_indices, sol, cb)
  temp.append([xB, xN, cb, cn, cost, reduced_costs, db, thetas[db<0], theta_star, k])

  # Update base_indices
  basis_indices.remove(exit_index)
  basis_indices.append(new_entry_index)
  basis_indices.sort() # ensure indices are in ascending order


pd.DataFrame(temp, columns=['xB', 'xN', 'cB', 'cN', 'Cost', 'Reduced Costs', 'dB', 'Thetas', 'Theta*', 'K']).set_index('K')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in true_divide


,xB,xN,cB,cN,Cost,Reduced Costs,dB,Thetas,Theta*
K,,,,,,,,,
1,"[x1=1.0, x2=3.0, x5=4.0]","[x3=0, x4=0]","[-2, 4, 0]","[0, 0]",10.0,"[-1.0, -3.0]","[-0.5, -0.5, 0.0]","[2.0, 6.0]",2.0
2,"[x2=2.0, x3=2.0, x5=4.0]","[x1=0, x4=0]","[4, 0, 0]","[-2, 0]",8.0,"[2.0, -4.0]","[-1.0, 1.0, -1.0]","[2.0, 4.0]",2.0
3,"[x3=4.0, x4=2.0, x5=2.0]","[x1=0, x2=0]","[0, 0, 0]","[-2, 4]",0.0,"[-2.0, 4.0]","[-1.0, 1.0, -1.0]","[4.0, 2.0]",2.0
4,"[x1=2.0, x3=2.0, x4=4.0]","[x2=0, x5=0]","[-2, 0, 0]","[4, 0]",-4.0,"[2.0, 2.0]",None,None,NaN
